In [15]:
from makedataset import makedataset
from root_numpy import array2root
from numpy.testing._private.utils import HAS_LAPACK64
from tensorflow.python.keras.saving.save import load_model
from tensorflow.python.keras.utils.vis_utils import plot_model

import numpy as np
from keras.layers import Dense
from keras.models import Model
from keras.layers import Input
from keras.layers import Reshape
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Concatenate
from keras.optimizers import Adam
import matplotlib.pyplot as plt

import os 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from keras import backend as K


In [2]:
data = makedataset('W2LNu10000Events_13Tev.root')

X_train = data.getGenFeatureVals()
y_train = data.getGenLabelVals()
X_star  = data.getRecoFeatureVals()

input_dim = X_train.shape[1]
print('-'*15,' dimensions of the input variables ','-'*15)
print('training   input dimensions',X_train.shape,'  ',y_train.shape)
print('predicitng input dimensions',X_star.shape)
print('-'*50)
num_pipeline = Pipeline([
    #('imputer',SimpleImputer(strategy="median")),
    ('std_scaler',StandardScaler()),
])

X_gen,X_reco = num_pipeline.fit_transform(X_train),num_pipeline.fit_transform(X_star)
M_gen  = num_pipeline.fit_transform(y_train.reshape(-1,1))

['geninvis_px', 'geninvis_py', 'geninvis_pz', 'geninvis_e', 'genmuon_px', 'genmuon_py', 'genmuon_pz', 'genmuon_e', 'muon_px', 'muon_py', 'muon_pz', 'muon_e', 'invis_px', 'invis_py', 'boson_M']
---------------  dimensions of the input variables  ---------------
training   input dimensions (10000, 8)    (10000,)
predicitng input dimensions (10000, 6)
--------------------------------------------------


In [17]:
import tensorflow as tf
class RecoVariables:
    def __init__(self,genData,recoData):
        self.genData  = genData
        self.recoData = recoData
        path_to_directory = self.setPath('default')
        print(f'plots are save {path_to_directory} directory')
        try:
            os.mkdir(path_to_directory)
        except:
            print('plots folder is already created')

        if(genData.shape[0] != recoData.shape[0]):
            print('datasets are not in same direction')
        
            
    def train(self,n_epoch,batch_size,sample_interval=10):
        half_batch = int(batch_size/2.0)
        self.ix = np.random.randint(0,self.gen_px.shape[0],half_batch)
        discriminator = self.build_discriminator(2)
        generator     = self.build_generator(1,1)
        gan           = self.build_gan(generator,discriminator)
        min_sep = 10**4
        # for i in range(1,n_epoch+1):
        #     xreal,yreal = self.generate_real_samples(half_batch)
        #     xfake,yfake = self.generate_fake_samples(generator,half_batch)
        #     discriminator.train_on_batch(xreal,yreal)
        #     discriminator.train_on_batch(xfake,yfake)
        #     [xgan,label_gan] = self.generate_latent_samples()
        #     ygan = np.ones((half_batch,1))
        #     gan.train_on_batch([xgan,label_gan],ygan)
        #     self.gan_px = self.eval(generator)
            

    def jointPzE(self,noise_kin):
        pz = noise_kin[:,-2]
        E  = np.sqrt(self.recoData[-2]**2 + self.recoData[-1]**2 + pz**2)
        new_kin = np.concatenate((self.recoDataset,pz.reshape(-1,1),E.reshape(-1,1)),axis=1)
        return new_kin
    
    def hist_area(self,hist,nbins):
        n,bins = np.histogram(hist,nbins)
        power = abs(bins[0]-bins[1])*sum(n)/2
        return power

    def predict_pz(self,gen_pz):
        g_model = load_model('ganGenerator.h5')
        pz = np.random.normal(0,0.4,gen_pz.shape[0])
        gen_input = [gen_pz,pz]
        gen_output = g_model.predict(gen_input)
        pred = gen_output-gen_pz.reshape(-1,1)
        return pred

    def eval(self,generator):
        self.noise = np.random.uniform(0,0.1,self.gen_px.shape[0])
        xgen  = generator.predict([self.gen_px,self.noise])
        return xgen-self.gen_px.reshape(-1,1)

    def generate_real_samples(self,n):
        xreal = np.concatenate((self.gen_px[self.ix].reshape(-1,1),self.reco_px[self.ix].reshape(-1,1)),axis=1)
        yreal = np.ones((n,1))
        return xreal,yreal

    def generate_latent_samples(self):
        noise = np.random.uniform(0,1,self.gen_px.shape[0])
        return [self.gen_px[self.ix],noise[self.ix]]

    def generate_fake_samples(self,generator,n):
        [xinput,zinput] = self.generate_latent_samples()
        xgen = generator.predict([xinput,zinput])
        xfake = np.concatenate((xinput.reshape(-1,1),xgen-xinput.reshape(-1,1)),axis=1)
        yfake = np.zeros((n,1))
        return xfake,yfake

    def setPath(self,path):
        if(path == 'default'):
            self.path = '/mnt/e/ML_massfit/plots/plots_fromPzProducer'
        else:
            self.path = path
        return self.path

    def build_generator(self,input_dim,output_dim):
        gen_input   = Input(shape=(input_dim,))
        y_input = Input(shape=(1,))
        merge = Concatenate()([gen_input,y_input])
        layer = Dense(64,activation='relu')(merge)
        layer = Dense(256,activation='relu')(layer)
        layer = Dense(64,activation='relu')(layer)
        out_layer = Dense(output_dim,activation='tanh')(layer)
        model = Model([gen_input,y_input],out_layer,name='autoencoder')
        plot_model(model,to_file=self.path+'/Generator_model_summary.png',
            show_shapes=True,show_layer_names=True)
        return model

    def build_discriminator(self,n_inputs):
        gen_output = Input(shape=(n_inputs,))
        y_output   = Input(shape=(1,))
        merge   = Concatenate()([gen_output,y_output])
        layer = Dense(128,activation='relu')(merge)
        layer = Dense(64,activation='relu')(layer)
        layer = Dense(32,activation='relu')(layer)
        out_layer = Dense(1,activation='sigmoid')(layer)
        model = Model([gen_output,y_output], out_layer)
        opt = Adam(lr=0.00002, beta_1=0.5)
        model.compile(loss='binary_crossentropy',
            optimizer=opt, metrics=['accuracy'])
        plot_model(model,to_file=self.path+'/Discriminator_model_summary.png',
            show_shapes=True,show_layer_names=True)
        return model

    def build_gan(self,generator,discriminator):
        discriminator.trainable = False
        input_gen,input_true = generator.input
        output_reco =  generator.output
        print(type(output_reco))
        out_array = K.eval(output_reco)
        reco_input = self.jointPzE(out_array)
        gen_output = [reco_input,input_true]
        gan_output = discriminator(gen_output)
        model = Model([input_gen,input_true],gan_output)
        opt = Adam(lr=0.00002, beta_1=0.5)
        model.compile(loss='binary_crossentropy', optimizer=opt)
        plot_model(model,to_file=self.path+'/GAN_model_summary.png',
            show_shapes=True,show_layer_names=True)
        return model


In [18]:
reco_obj = RecoVariables(X_gen,X_reco)

plots are save /mnt/e/ML_massfit/plots/plots_fromPzProducer directory
plots folder is already created


In [19]:
discriminator = reco_obj.build_discriminator(8)
generator     = reco_obj.build_generator(8,8)
gan = reco_obj.build_gan(generator,discriminator)

/home/vinay/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


<class 'keras.engine.keras_tensor.KerasTensor'>


AttributeError: 'KerasTensor' object has no attribute 'numpy'

In [14]:
filename = '../data/W2LNu10000Events_13Tev_new.root'
leaf = zip(recopz,recoE)
leaf_l = list(leaf)
branch = np.array(leaf_l,dtype=[('invis_pz','float32'),('invis_e','float32')])
array2root(branch,filename,treename='analysisTree',mode='update')

In [20]:
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

In [21]:
type(train_images)

numpy.ndarray

In [22]:
train_labels = tf.one_hot(train_labels, depth=10)
type(train_labels)

tensorflow.python.framework.ops.EagerTensor

In [29]:
te = tf.constant([1,2,3],dtype=tf.float32)

In [30]:
tf.expand_dims(te,axis=1)

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[1.],
       [2.],
       [3.]], dtype=float32)>

In [25]:
te[:,0]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 4., 7.], dtype=float32)>